## 1. Importation des bibliothèque 

In [1]:
# Analyse de données
import pandas as pd
import numpy as np

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter uniquement :  afficher les objets de manière interactive et mieux formatée que le simple print()
from IPython.display import display   

# Configurations optionnelles
pd.set_option('display.max_columns', None)              
pd.set_option('display.float_format', '{:.2f}'.format)

#Création du modele logistique
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## 2.Chargement des données

In [2]:
df = pd.read_csv(r"C:\Users\zineh\Desktop\Formation Data Analyse\Dossier module 6- PFE\Output\University Learning Analytics Dataset.csv")

#### Traduction du dataset en français pour avoir un jeu de données plus clair et compréhensible

In [3]:
# Traduction des colonnes
df = df.rename(columns={
    "id_student": "id_etudiant",
    "gender": "genre",
    "region": "region",
    "highest_education": "niveau_education",
    "age_band": "tranche_age",
    "num_of_prev_attempts": "tentatives_precedentes",
    "studied_credits": "credits_etudies",
    "disability": "handicap",
    "final_result": "resultat_final",
    "date_registration": "date_inscription",
    "date_unregistration": "date_desinscription",
    "avg_score": "moyenne_notes",
    "total_clicks": "total_clics",
    "nb_resources": "nb_ressources"
   })

# Traduction des valeurs textuelles
df["genre"] = df["genre"].replace({"M": "Homme", "F": "Femme"})
df["handicap"] = df["handicap"].replace({"Y": "Oui", "N": "Non"})
df["resultat_final"] = df["resultat_final"].replace({
    "Pass": "Réussi",
    "Fail": "Échoué",
    "Withdrawn": "Abandon",
    "Distinction": "Mention"
})
df["niveau_education"] = df["niveau_education"].replace({
    "Post Graduate Qualification": "Diplôme de 3e cycle",
    "HE Qualification": "Diplôme universitaire",
    "A Level or Equivalent": "Bac ou équivalent",
    "Lower Than A Level": "Inférieur au Bac",
    "No Formal quals": "Sans diplôme"
})

print(df.head())

   id_etudiant  genre               region       niveau_education tranche_age  \
0         3733  Homme         South Region  Diplôme universitaire        55<=   
1         6516  Homme             Scotland  Diplôme universitaire        55<=   
2         8462  Homme        London Region  Diplôme universitaire        55<=   
3         8462  Homme        London Region  Diplôme universitaire        55<=   
4        11391  Homme  East Anglian Region  Diplôme universitaire        55<=   

   tentatives_precedentes  credits_etudies handicap resultat_final  \
0                       0               60      Non        Abandon   
1                       0               60      Non         Réussi   
2                       0               90      Non        Abandon   
3                       1               60      Non        Abandon   
4                       0              240      Non         Réussi   

   date_inscription  date_desinscription  moyenne_notes  total_clics  \
0            -68.00 

In [4]:
df.head().round(2)

,id_etudiant,genre,region,niveau_education,tranche_age,tentatives_precedentes,credits_etudies,handicap,resultat_final,date_inscription,date_desinscription,moyenne_notes,total_clics,nb_ressources
0,3733,Homme,South Region,Diplôme universitaire,55<=,0,60,Non,Abandon,-68.00,-8.00,NaN,NaN,0
1,6516,Homme,Scotland,Diplôme universitaire,55<=,0,60,Non,Réussi,-52.00,NaN,61.80,NaN,0
2,8462,Homme,London Region,Diplôme universitaire,55<=,0,90,Non,Abandon,-137.00,119.00,NaN,NaN,0
3,8462,Homme,London Region,Diplôme universitaire,55<=,1,60,Non,Abandon,-38.00,18.00,NaN,NaN,0
4,11391,Homme,East Anglian Region,Diplôme universitaire,55<=,0,240,Non,Réussi,-159.00,NaN,82.00,3140.00,39


## 3. Analayse exploratoire des données (EDA)

### Information générale

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id_etudiant             1000 non-null   int64  
 1   genre                   1000 non-null   object 
 2   region                  1000 non-null   object 
 3   niveau_education        1000 non-null   object 
 4   tranche_age             1000 non-null   object 
 5   tentatives_precedentes  1000 non-null   int64  
 6   credits_etudies         1000 non-null   int64  
 7   handicap                1000 non-null   object 
 8   resultat_final          1000 non-null   object 
 9   date_inscription        997 non-null    float64
 10  date_desinscription     315 non-null    float64
 11  moyenne_notes           439 non-null    float64
 12  total_clics             45 non-null     float64
 13  nb_ressources           1000 non-null   int64  
dtypes: float64(4), int64(4), object(6)
memory

### Nettoyage des données

In [6]:
# Remplacer les valeurs manquantes dans la colonne "date_inscription" 
# par "0" (hypothèse : inscription le jour même du début du cours)

df['date_inscription'] = df['date_inscription'].fillna(0)

In [7]:
# Créer une nouvelle colonne etat_inscription avec inscription_maintenue, désinscription_avancée si desinscription <30jrs 
# et desinscription tardive si desinscription >30 jrs après début de la formation

def etat_inscription(jours):
    if pd.isna(jours):
        return 'inscription_maintenue'  # Pas de désinscription
    elif jours < 30:
        return 'désinscription_avancée'
    else:
        return 'désinscription_tardive'
        
df['etat_inscription'] = df['date_desinscription'].apply(etat_inscription)

#Supprimer la colonne date_desinscription

df.drop('date_desinscription', axis=1, inplace=True)

In [8]:
# Remplacer les NAN par 0 dans la colonne total_clics et dans la colonne moyenne_notes

df["total_clics"] = df["total_clics"].fillna(0)
df["moyenne_notes"] = df["moyenne_notes"].fillna(0)

In [9]:
# Traitement de la colonne tranche_age
df['tranche_age'] = df['tranche_age'].replace({'0-35': '<=35'})
df['tranche_age'] = df['tranche_age'].replace({'55<=': '>=55'})

In [10]:
#Créer une nouvelle colonne abondon pour la prédiction 

df['abandon'] = df['resultat_final'].apply(lambda x: 1 if x == 'Abandon' else 0)

# Vérifier le résultat
print(df[['resultat_final', 'abandon']].head())


  resultat_final  abandon
0        Abandon        1
1         Réussi        0
2        Abandon        1
3        Abandon        1
4         Réussi        0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id_etudiant             1000 non-null   int64  
 1   genre                   1000 non-null   object 
 2   region                  1000 non-null   object 
 3   niveau_education        1000 non-null   object 
 4   tranche_age             1000 non-null   object 
 5   tentatives_precedentes  1000 non-null   int64  
 6   credits_etudies         1000 non-null   int64  
 7   handicap                1000 non-null   object 
 8   resultat_final          1000 non-null   object 
 9   date_inscription        1000 non-null   float64
 10  moyenne_notes           1000 non-null   float64
 11  total_clics             1000 non-null   float64
 12  nb_ressources           1000 non-null   int64  
 13  etat_inscription        1000 non-null   object 
 14  abandon                 1000 non-null   i

In [12]:
df.head()

,id_etudiant,genre,region,niveau_education,tranche_age,tentatives_precedentes,credits_etudies,handicap,resultat_final,date_inscription,moyenne_notes,total_clics,nb_ressources,etat_inscription,abandon
0,3733,Homme,South Region,Diplôme universitaire,>=55,0,60,Non,Abandon,-68.00,0.00,0.00,0,désinscription_avancée,1
1,6516,Homme,Scotland,Diplôme universitaire,>=55,0,60,Non,Réussi,-52.00,61.80,0.00,0,inscription_maintenue,0
2,8462,Homme,London Region,Diplôme universitaire,>=55,0,90,Non,Abandon,-137.00,0.00,0.00,0,désinscription_tardive,1
3,8462,Homme,London Region,Diplôme universitaire,>=55,1,60,Non,Abandon,-38.00,0.00,0.00,0,désinscription_avancée,1
4,11391,Homme,East Anglian Region,Diplôme universitaire,>=55,0,240,Non,Réussi,-159.00,82.00,3140.00,39,inscription_maintenue,0


### Analyse describtive

In [13]:
df.describe()

,id_etudiant,tentatives_precedentes,credits_etudies,date_inscription,moyenne_notes,total_clics,nb_ressources,abandon
count,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00
mean,81555.97,0.34,83.05,-71.99,30.45,201.03,1.93,0.32
std,32624.39,0.71,44.01,50.85,36.58,1097.66,9.31,0.47
min,3733.00,0.00,30.00,-284.00,0.00,0.00,0.00,0.00
25%,54306.50,0.00,60.00,-103.75,0.00,0.00,0.00,0.00
50%,82271.00,0.00,60.00,-58.00,0.00,0.00,0.00,0.00
75%,110181.25,0.00,120.00,-31.00,70.85,0.00,0.00,1.00
max,132157.00,4.00,430.00,82.00,100.00,11745.00,67.00,1.00


In [14]:
# Analyse de la distribution des tranches d'âge
df["tranche_age"].value_counts()

tranche_age
<=35     632
35-55    352
>=55      16
Name: count, dtype: int64

In [15]:
# Analyse de la distribution du genre
df["genre"].value_counts()

genre
Homme    555
Femme    445
Name: count, dtype: int64

In [16]:
# Analyse de la distribution du resultat final
df["resultat_final"].value_counts()

resultat_final
Réussi     394
Abandon    319
Échoué     208
Mention     79
Name: count, dtype: int64

In [17]:
# Analyse de la distribution de l'handicap
df["handicap"].value_counts()

handicap
Non    878
Oui    122
Name: count, dtype: int64

In [18]:
# Analyse de la distribution de l'état_inscription
df["etat_inscription"].value_counts()

etat_inscription
inscription_maintenue     685
désinscription_tardive    174
désinscription_avancée    141
Name: count, dtype: int64

In [19]:
#Export du fichier pour analyse et dashboard 
df.to_excel("Analyse_abandon.xlsx", index=False)
print("\n Fichier 'Analyse_abandon.xlsx' est généré avec succès !")


 Fichier 'Analyse_abandon.xlsx' est généré avec succès !


##  Prédiction de l'abandon

In [20]:
# Analyse de la distribution de l'abandons
df["abandon"].value_counts()

abandon
0    681
1    319
Name: count, dtype: int64

Répartition de l'abandon (N=1000)

681 inscriptions maintenues (68%) et 319 abandons (32%)

Le taux d'abandon est important et nécessite une action immédiate 

## 1. Sélection des Features et de la Target

In [21]:
# 1) Définir X,y sans les fuites
drop_cols = ['abandon', 'resultat_final', 'etat_inscription', 'id_etudiant']  
X = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')
y = df['abandon']

## 2. Split train/test

In [22]:
# Diviser les données en train et test avec 80% entrainement et 20% test
# et random_state=42 → rend le découpage reproductible : on aura toujours les mêmes données dans le train/test à chaque exécution

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Séparer types
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(include=['object']).columns.tolist()

# Préprocesseur
preproc = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
    ]
)


 ## 3. Entrainement du modèle 

In [23]:
pipe = Pipeline([
    ('preproc', preproc),
    ('clf', LogisticRegression(max_iter=1000))
])

pipe.fit(X_train, y_train)


Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['tentatives_precedentes',
                                                   'credits_etudies',
                                                   'date_inscription',
                                                   'moyenne_notes',
                                                   'total_clics',
                                                   'nb_ressources']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['genre', 'region',
                                                   'niveau_education',
                                                   'tranche_age',
                                                   'handicap'])])),
                ('clf', LogisticRegression(max_iter=1000))])

## 4. Evaluation du modèle

In [24]:
pred = pipe.predict(X_test)                              # pour faire des prédictions sur l'ensemble de test (X_test)
accuracy = accuracy_score(y_test, pred)                  # calcule la précision du modèle(% de prédictions correctes par rapport à l'ensemble de test)
print(f"Précision du modèle: {accuracy*100:.2f}%")         


Précision du modèle: 72.50%


##### Le modèle prédit correctement l'abandon des étudiants dans 72.50% des cas

In [25]:
# Création de la matrice de confusion 
print('Matrice de confusion :')
print(confusion_matrix(y_test, pred))
print('Rapport de classification :')
print(classification_report(y_test, pred, digits=3))   


Matrice de confusion :
[[127   9]
 [ 46  18]]
Rapport de classification :
              precision    recall  f1-score   support

           0      0.734     0.934     0.822       136
           1      0.667     0.281     0.396        64

    accuracy                          0.725       200
   macro avg      0.700     0.608     0.609       200
weighted avg      0.713     0.725     0.686       200



#### Lecture de la matrice de confusion

Vrais négatifs (TN) = 127 : Le modèle a correctement prédit 127 cas où les étudiants n'ont pas abandonné. Cela signifie que pour 127 fois, le modèle a correctement identifié que les étudaiants n'ont pas abandonné.

Faux positifs (FP) = 9 : Le modèle a incorrectement prédit 9 cas comme étant des abandons qu'en réalité, ils n'ont pas abandonné. Cela veut dire que le modèle a prédit à tort que ces 9 étudiants qui ont abandonnée.

Faux négatifs (FN) = 46: Le modèle a incorrectement prédit 46 cas comme n'étant pas des abandons alors qu'en réalité, ils étaient des abandons. Autrement dit, le modèle a manqué 46 abandons réels, pensant à tort que ces étudiants n'ont pas abandonné.

Vrais positifs (TP) = 18 : Le modèle a correctement prédit 18 cas d4ABANDON. Cela signifie que pour 18 fois, le modèle a correctement identifié que les étudiants abandonneraient.

#### Interpretation de la matrice de confusion

Le modèle est plutôt bon pour identifier les abandons : Avec 127 vrais positifs, le modèle est capable de détecter un grand nombre d'abandon réel

Le modèle a une marge d'amélioration pour réduire les erreurs : Les 09 faux positifs et 46 faux négatifs montrent que le modèle fait des erreurs tant en prévoyant des abandons inexistants qu'en manquant des abandons réels

Équilibre entre la sensibilité et la spécificité : Le modèle semble avoir un équilibre entre la capacité à détecter les abandons (sensibilité) et la capacité à identifier correctement les non-abandons (spécificité), même si l'amélioration est possible dans les deux domaines

## 5. Prédictions : Création de la fonction predict_abandon()

In [26]:
display(df.columns)

Index(['id_etudiant', 'genre', 'region', 'niveau_education', 'tranche_age',
       'tentatives_precedentes', 'credits_etudies', 'handicap',
       'resultat_final', 'date_inscription', 'moyenne_notes', 'total_clics',
       'nb_ressources', 'etat_inscription', 'abandon'],
      dtype='object')

In [27]:
def predict_abandon(pipe, X_columns, values):
    """
    pipe       : pipeline entraîné (avec encodage + modèle)
    X_columns  : la liste des colonnes de X (dans le même ordre qu'à l'entraînement)
    values     : un dictionnaire {colonne: valeur} pour l'étudiant à prédire
    """
    
    # Créer un DataFrame avec une seule ligne, et toutes les colonnes attendues
    input_df = pd.DataFrame([values], columns=X_columns)
    
    # Prédire la probabilité d'abandon avec le pipeline
    proba_abandon = pipe.predict_proba(input_df)[0][1]
    
    return f"Probabilité d'abandon : {proba_abandon * 100:.2f}%"


##### Exemple d'utilisation

In [28]:
 #Récupération des colonnes utilisées pour entraîner le pipeline
X_columns = X.columns

# Appel de la fonction avec des valeurs catégorielles correctes
etudiant = {
    'genre': 'Femme',  
    'region': 'London',  
    'niveau_education': 'Diplôme universitaire',  
    'tranche_age': '>=55',  
    'tentatives_precedentes': 1,
    'credits_etudies': 130,
    'handicap': 'Oui',  
    'date_inscription': 30,
    'moyenne_notes': 56,
    'total_clics': 100,
    'nb_ressources': 30
}

print(predict_abandon(pipe, X_columns, etudiant))


Probabilité d'abandon : 58.23%


Un étudiant de sexe feminin déclarant un handicap dont l'âge est supérieur ou égale à 55 ans, résident à London doté d'un diplôme universitaire, ayant une tentative 

d'inscription précédente. Inscrit depuis 30 jours, avec un crédit étudié de 130, un score académique de 56 et une activité sur la plateforme de 100 clics au total et 30 

ressources ; a une probabilité de 58.23% d'abandonner la formation en e-learning 

In [30]:
# df = DataFrame avec toutes les données des étudiants
# X_columns = liste des colonnes utilisées pour le modèle
# pipe = pipeline entraîné

# Fonction pour calculer la probabilité pour une ligne
def calc_proba(row):
    values = {col: row[col] for col in X_columns}
    return pipe.predict_proba(pd.DataFrame([values], columns=X_columns))[0][1]

# Appliquer la fonction à chaque étudiant
df['proba_abandon'] = df.apply(calc_proba, axis=1)

# Créer le DataFrame pour Power BI
predictions_df = df[['id_etudiant', 'proba_abandon']]

# Sauvegarder en excel
predictions_df.to_excel('predictions_abandon.xlsx', index=False)